In [88]:
import pandas as pd
from pathlib import Path
import os
import requests
import json
from MCForecastTools import MCSimulation
import hvplot.pandas
import bokeh
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [85]:
currency_data = pd.read_csv(
    Path("../Project-1-Currencies/data/Currency-Data-1-Collated.csv"), 
    index_col= "Year", 
    parse_dates=True, 
    infer_datetime_format=True
)

In [86]:
currency_data.head()

,USD,CAD,MXN,BRL,GBP,CHF,RUB,NOK,EUR,TRY,SAR,ZAR,AUD,JPY,KRW,CNY,INR
Year,,,,,,,,,,,,,,,,,
1950-01-01,1,1.089,0.00865,0.0,0.357143,4.37295,0.0,7.14286,0.0,2.8,0.00,0.714286,0.892857,361.1,2.5,0.0,4.7619
1951-01-01,1,1.053,0.00865,0.0,0.357143,4.37295,0.0,7.14286,0.0,2.8,0.00,0.714286,0.892857,361.1,6.0,0.0,4.7619
1952-01-01,1,0.979,0.00865,0.0,0.357143,4.37295,0.0,7.14286,0.0,2.8,3.75,0.714286,0.892857,361.1,6.0,0.0,4.7619
1953-01-01,1,0.983,0.00865,0.0,0.357143,4.37295,0.0,7.14286,0.0,2.8,3.75,0.714286,0.892857,360.0,18.0,0.0,4.7619
1954-01-01,1,0.973,0.01250,0.0,0.357143,4.37295,0.0,7.14286,0.0,2.8,3.75,0.714286,0.892857,360.0,18.0,0.0,4.7619


In [87]:
currency_data.hvplot.line()

:NdOverlay   [Variable]
   :Curve   [Year]   (value)

In [62]:
interest_rates = pd.read_csv(
    Path("../Project-1-Currencies/data/Interest-Rate-Data.csv"), 
    index_col= "Year", 
    parse_dates=True, 
    infer_datetime_format=True
)

In [63]:
interest_rates.hvplot.line()

:NdOverlay   [Variable]
   :Curve   [Year]   (value)

In [10]:
CADdata = currency_data.iloc[:, [1]]

CADdata.head()

,CAD
Year,
1950-01-01,1.089
1951-01-01,1.053
1952-01-01,0.979
1953-01-01,0.983
1954-01-01,0.973


In [65]:
import nasdaqdatalink
import pandas as pd
mydata = nasdaqdatalink.get("BOE/XUDLADD")
mydata1 = nasdaqdatalink.get("BOE/XUDLADD", transformation="diff")
mydata_final = pd.concat([mydata, mydata1], axis=1)
#mydata_final["Date"] = pd.to_datetime(mydata_final["Date"])
mydata_final

,Value,Value
Date,,
1975-01-02,0.7540,NaN
1975-01-03,0.7552,0.0012
1975-01-06,0.7537,-0.0015
1975-01-07,0.7518,-0.0019
1975-01-08,0.7520,0.0002
...,...,...
2022-10-12,1.5949,0.0038
2022-10-13,1.6082,0.0133
2022-10-14,1.6023,-0.0059


In [89]:
#currency_data = pd.read_csv(
#    Path("../ratedata12.csv"), 
 #   index_col= "timestamp", 
 #   parse_dates=True, 
  #  infer_datetime_format=True
#)
#currency_data

In [77]:
# Set timeframe to '1Day'
timeframe = "1Day"

# Set start and end datetimes between now and 3 years ago
start_date = pd.Timestamp("1975-01-02", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-08", tz="America/New_York").isoformat()

# Get 3 years’ worth of historical price data for Microsoft and Coca-Cola
prices_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

,close,daily_return
timestamp,,
2020-10-01,100,0
2020-10-02,101,1
2020-10-03,103,2
2020-10-04,105,2
2020-10-05,108,3


In [71]:
from MCForecastTools import MCSimulation
MC_even_weight = MCSimulation(
    portfolio_data = mydata,
    num_simulation = 1000,
    num_trading_days = 25
)

IndexError: Too many levels: Index has only 1 level, not 2

In [ ]:
sns.set()
plt.ylabel('CAD in relation to USD')
plt.xlabel('year')
plt.xticks(rotation=45)

plt.plot(CADdata, )

In [ ]:
train = CADdata[CADdata.index < pd.to_datetime("2018")]
test = CADdata[CADdata.index > pd.to_datetime("2018")]

plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.ylabel('CAD to USD')
plt.xlabel('Year')
plt.xticks(rotation=45)
plt.title("Train/Test split for BTC Data")
plt.show()

from statsmodels.tsa.statespace.sarimax import SARIMAX
y = train[" CAD to USD "]
ARMAmodel = SARIMAX(y, order = (1, 0, 1))
ARMAmodel = ARMAmodel.fit()

y_pred = ARMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"] 

plt.plot(y_pred_out, color='green', label = 'Predictions')
plt.legend()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
y = train['CADdata']
ARMAmodel = SARIMAX(y, order = (1, 0, 1))
ARMAmodel = ARMAmodel.fit()

y_pred = ARMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"] 

plt.plot(y_pred_out, color='green', label = 'Predictions')
plt.legend()

In [15]:
# Import the required libraries and dependencies
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import json

# Load .env environment variables
load_dotenv('../Nasdaqkey.env')

True

In [19]:
# Retrieve the API key from the environment variable and store as Python variable
nasdaq_api_key = os.getenv("NASDAQ_API_KEY")
type(nasdaq_api_key)

# Define the base request URL for NASDAQ
# Note the `limit=10` and api_key=` parameters.
request_url = "https://data.nasdaq.com/api/v3/datasets/BOE/IUDGR6M.csv?start_date=2014-01-01&end_date=2014-12-31&api_key=YOURAPIKEY"
#"https://data.nasdaq.com/api/v3/datasets/WIKI/MSFT.json?limit=10&api_key="

# Concatenate request_url and nasdaq_api_key and store as a new variable
nasdaq_request_url = request_url + nasdaq_api_key

# Execute the API call to Nasdaq using requests.get
response = requests.get(nasdaq_request_url).json()

# Display response data using json.dumps
print(json.dumps(response, indent=4, sort_keys=True))

JSONDecodeError: [Errno Expecting value] code,message
"QELx06","You have exceeded the API speed limit and your account has temporaly been disabled.  Please contact clientsuccess@nasdaq.com for more information.": 0

In [14]:
# Set the ticker information
tickers = ["MSFT","KO"]

# Set timeframe to '1Day'
timeframe = "1Day"

# Set start and end datetimes between now and 3 years ago
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()

# Get 3 years’ worth of historical price data for Microsoft and Coca-Cola
prices_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Reorganize the DataFrame
# Separate ticker data
MSFT = prices_df[prices_df['symbol']=='MSFT'].drop('symbol', axis=1)
KO = prices_df[prices_df['symbol']=='KO'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
prices_df = pd.concat([MSFT, KO], axis=1, keys=["MSFT", "KO"])

# Review the first 5 rows of the DataFrame
prices_df.head()

NameError: name 'alpaca' is not defined

In [ ]:
# Import the MCSimulation module from the MCForecastTools library file
from MCForecastTools import MCSimulation

# Print the documentation of the MCSimulation module of the MCForecastTools library
?MCSimulation

In [ ]:
MC_fiveyear = MCSimulation(
    portfolio_data = prices_df,
    weights = [.60,.40],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Run a Monte Carlo simulation to forecast five years cumulative returns
MC_fiveyear.calc_cumulative_return()